DOI: https://github.com/wri/global-pasture-watch/tree/main/ggc-30m

O escopo de trabalho consiste em rodar uma série de análises de sensibilidade utilizando as amostras do Global Pasture Watch, especificamente o arquivo gpw_grassland_fscs.vi.vhr.overlaid_point.samples_20000101_20221231_go_epsg.4326_v1.pq (disponível em https://doi.org/10.5281/zenodo.13952806 & Github).
As amostras consistem em um conjunto de pontos, agrupados/clusterizados por regiões/tiles de 1 km, indicadas pela coluna vi_tile_id (para mais informações ver https://doi.org/10.21203/rs.3.rs-4514820/v3)
Segue abaixo as informações para as análises:
- Para fins de comparação, recomendo que você selecione aleatoriamente 20% dos tiles 1-km (aprox. 460,000 pontos), de forma que os 80% dos tiles que sobrarem sejam utilizados para calibração e treinamento dos modelos de ML. Utilize os 20% como conjunto de teste independente, garantindo uma comparação robusta e em sem viés.
- Para todas as análise assuma como modelo baseline o RF treinando com os hyper-parameteros do GPW. Esse foi o modelo que utilizamos para gerar o mapas de grassland v1. Todas as comparações devem includir F1-Score, log_loss, recall_score & precision_score por classe e geral.
- Para rodar as análises com esse volume de dados você precisará ter acesso a um servidor robusto (64 GB of RAM & >32 threads) Veja com o @Laerte Ferreira quem pode lhe ajudar com isso no LAPIG. 

Segue abaixo as análises a serem implementadas:

- Treinar um modelo LightGBM (com early_stopping) e comparar com o modelo baseline 
- Treinar um modelo RF para cada continente e comparar com o modelo baseline. Nessa análise, você precisa adicionar uma coluna continente no conjunto de treinamento e teste (veja geopandas.sjoin). Para o conjunto de teste, a coluna continente deve ser utilizada para encontrar o modelo local previamente treinado
- Treinar vários modelos RF aumentando graduamente o número de amostras em cada tile (10, 25, 50, 75 e 100%) e comparar com o modelo baseline
- Treinar um modelo RF com número de amostras iguais para todas as 3 classes (ex: exatamente 200,000 amostras para 1-cultivated grassland, 2-natural/semi-natural grassland e 3-others) e comparar com o modelo baseline.
- Treinar um modelo RF removendo outliers (Ex: Novelty and Outlier Detection) e comparar com o modelo baseline.

Análises adicionais propostas por você são bem-vindas :)

Remoção de Outliers:

- pyod
- isolation forest

Link: https://scikit-learn.org/1.5/modules/outlier_detection.html

# Carregar Dados

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

SAMPLES_FILE_PATH = "c_samples.pq"

In [2]:
def get_samples():
    samples = pd.read_parquet(SAMPLES_FILE_PATH)

    return samples.reset_index()

# Misc

In [3]:
import os

class PathHandler():
    __path: str = ''
    __value: str = ''
    
    @classmethod
    def generate_path(cls, file_name: str):
        return f'{cls.__path}/{file_name}_{cls.__value}.lz4'
    
    @classmethod
    def get_path(cls):
        return cls.__path
    
    @classmethod
    def set_path(cls, path: str):
        os.makedirs(path, exist_ok=True)
        
        PathHandler.__path = path
        
    @classmethod
    def set_value(cls, value: str):
        cls.__value = value

In [4]:
import time

def measure_execution_time(func):
    def wrapper(*args, **kwargs):
        begin = time.time()
        resultado = func(*args, **kwargs)
        end = time.time()
        print('\n\n' + f'| Tempo de execução de {func.__name__}: {end - begin:.4f} segundos |'.center(200, '-') + '\n\n')
        return resultado
    return wrapper

# Trainamento do Modelo

## Importando Bibliotecas

In [5]:
from pathlib import Path
import multiprocessing
import sys

from scipy.signal import argrelmin
from scipy.stats import uniform, randint

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_selection import RFECV, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import HalvingRandomSearchCV, GroupKFold, KFold
from sklearn.metrics import precision_score, recall_score, f1_score

import multiprocessing

import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Processamento

In [6]:
COVARIATE_START_COLUMN = 'ml_type'
SPATIAL_CROSS_VALIDATION_COLUMN = 'ml_cv_group'

CROSS_VALIDATION_NJOBS, CROSS_VALIDATION_FOLDS = 5, 5

TARGET_COLUMN = 'class'

RANDOM_STATE = 1989

In [7]:
def get_covariates(samples: pd.DataFrame):
    covariates = samples.columns
    
    return samples.columns[np.logical_or.reduce([
        covariates.str.contains('accessibility'),
        covariates.str.contains('blue'),
        covariates.str.contains('bsf'),
        covariates.str.contains('bsi'),
        covariates.str.contains('clm'),
        covariates.str.contains('dtm'),
        covariates.str.contains('evi'),
        covariates.str.contains('fapar'),
        covariates.str.contains('green'),
        covariates.str.contains('ndti'),
        covariates.str.contains('ndvi'),
        covariates.str.contains('ndwi'),
        covariates.str.contains('nir'),
        covariates.str.contains('nirv'),
        covariates.str.contains('red'),
        covariates.str.contains('road.distance_osm.highways.high.density'),
        covariates.str.contains('road.distance_osm.highways.low.density'),
        covariates.str.contains('swir1'),
        covariates.str.contains('swir2'),
        covariates.str.contains('thermal'),
        covariates.str.contains('water.distance_glad.interanual.dynamic.classes'),
        covariates.str.contains('wv_mcd19a2v061')
    ])]

In [8]:
def target_ovo(samples: pd.DataFrame, class_name: str, class_a: list[int], class_b: list[int]):
    remap_dict = {}
    
    remap_dict.update({val: 0 for val in class_a})
    remap_dict.update({val: 1 for val in class_b})
    
    samples[class_name] = samples[TARGET_COLUMN].map(remap_dict)


def create_ovo_class(samples: pd.DataFrame, class_name: list[str], class_values: list[tuple[list[int], list[int]]]):
    class_data = dict(zip(class_name, class_values))
    
    for class_key in class_data:
        value_a = class_data[class_key][0]
        value_b = class_data[class_key][1]
        
        target_ovo(samples, class_key, value_a, value_b)

In [9]:
def get_optimal_threshold(y_true: pd.DataFrame, y_pred):
    precision, recall, threshold = precision_recall_curve(y_true, y_pred)
    
    nonzero_mask = np.logical_and((precision != 0.0), (recall != 0.0))
    
    optimal_idx = np.argmax(1 - np.abs(precision[nonzero_mask] - recall[nonzero_mask]))
    
    return threshold[optimal_idx]

In [10]:
def get_estimator():
    return RandomForestClassifier(n_estimators=60, n_jobs=-1)


def random_forest(samples: pd.DataFrame, target_column: str, covariates: list[str], continent: str):
    tc_samples = samples[np.logical_not(np.isnan(samples[target_column]))]

    X = tc_samples[tc_samples['continent'] == continent][covariates]
    y = tc_samples[tc_samples['continent'] == continent][target_column]

    groupKFold = GroupKFold(CROSS_VALIDATION_FOLDS)

    y_true = []

    b_pred_proba = []
    c_pred_proba = []

    for train_idx, test_idx in groupKFold.split(X, y, tc_samples[tc_samples['continent'] == continent][SPATIAL_CROSS_VALIDATION_COLUMN]):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        B_train = tc_samples[~tc_samples.index.isin(X_test.index)][covariates]
        b_train = tc_samples[~tc_samples.index.isin(X_test.index)][target_column]

        b_estimator = get_estimator()
        c_estimator = get_estimator()
        b_estimator.fit(B_train, b_train)
        c_estimator.fit(X_train, y_train)

        y_true.extend(list(y_test))
        b_pred_proba.extend(b_estimator.predict_proba(X_test)[:,1])
        c_pred_proba.extend(c_estimator.predict_proba(X_test)[:,1])

    b_op_threshold = get_optimal_threshold(y_true, b_pred_proba)
    c_op_threshold = get_optimal_threshold(y_true, c_pred_proba)

    b_pred = (b_pred_proba >= b_op_threshold).astype(int)
    c_pred = (c_pred_proba >= c_op_threshold).astype(int)

    joblib.dump({
        'cv_result': pd.DataFrame({
            'baseline_predict_proba': b_pred_proba,
            'continent_predict_proba': c_pred_proba,
            'expected': y.to_numpy(),
        }),
        'baseline': {
            'threshold': b_op_threshold,
            'recall': recall_score(y_true, b_pred),
            'precision': precision_score(y_true, b_pred),
            'f1_score': f1_score(y_true, b_pred),
        },
        'continent': {
            'threshold': c_op_threshold,
            'recall': recall_score(y_true, c_pred),
            'precision': precision_score(y_true, c_pred),
            'f1_score': f1_score(y_true, c_pred),
        }
    }, PathHandler.generate_path('model'), compress='lz4')

# Treinamento de Modelos

In [ ]:
class_name = ['other_vs_cultivated', 'other_vs_natural']
class_values = [([3], [1]), ([3], [2])]

for continent in ['Asia', 'North America', 'South America', 'Africa', 'Europe', 'Australia', 'Oceania']:
    PathHandler.set_path(f'random_forest/continents')
    
    samples = get_samples()

    covariates = pd.read_csv('covariates.csv')['covariates']

    create_ovo_class(samples, class_name, class_values)

    for target_column in class_name:
        b_estimator = joblib.load(f"random_forest/baseline/model_{target_column}.lz4")['model']

        PathHandler.set_value(f"{continent.lower().replace(' ', '_')}_" + target_column)
        
        random_forest(samples, target_column, covariates, continent)

# END